In [1]:
import itertools
import math

from game import Cage, op, Cell, Puzzle
import game_parser

In [2]:
puzzle = game_parser.get_puzzle(98683)
puzzle

Puzzle(board_size=8, cages=[Cage(cells=[Cell(row=0, col=0), Cell(row=1, col=0)], op=<op.SUB: '-'>, val=7), Cage(cells=[Cell(row=0, col=1), Cell(row=0, col=2)], op=<op.PROD: '*'>, val=24), Cage(cells=[Cell(row=0, col=3), Cell(row=1, col=3), Cell(row=1, col=2)], op=<op.SUM: '+'>, val=18), Cage(cells=[Cell(row=0, col=4), Cell(row=0, col=5)], op=<op.DIV: '/'>, val=3), Cage(cells=[Cell(row=0, col=6), Cell(row=0, col=7)], op=<op.PROD: '*'>, val=10), Cage(cells=[Cell(row=1, col=1), Cell(row=2, col=1), Cell(row=2, col=0)], op=<op.PROD: '*'>, val=105), Cage(cells=[Cell(row=1, col=4), Cell(row=1, col=5)], op=<op.DIV: '/'>, val=3), Cage(cells=[Cell(row=1, col=6), Cell(row=2, col=6)], op=<op.SUM: '+'>, val=6), Cage(cells=[Cell(row=1, col=7), Cell(row=2, col=7)], op=<op.SUB: '-'>, val=2), Cage(cells=[Cell(row=2, col=2), Cell(row=2, col=3)], op=<op.PROD: '*'>, val=24), Cage(cells=[Cell(row=2, col=4), Cell(row=2, col=5)], op=<op.SUM: '+'>, val=15), Cage(cells=[Cell(row=3, col=0), Cell(row=3, col=1)],

In [3]:
# Pretty-print the cages for a better look at them

def cell_str(cell: Cell) -> str:
    return f'({cell.row},{cell. col})'

def cage_str(cage: Cage) -> str:
    s =  f'{cage.op.value}{cage.val:3}   {len(cage.cells)} cells: {" ".join([cell_str(c) for c in cage.cells])}'
    return s

[print(cage_str(c)) for c in puzzle.cages]
None


-  7   2 cells: (0,0) (1,0)
* 24   2 cells: (0,1) (0,2)
+ 18   3 cells: (0,3) (1,3) (1,2)
/  3   2 cells: (0,4) (0,5)
* 10   2 cells: (0,6) (0,7)
*105   3 cells: (1,1) (2,1) (2,0)
/  3   2 cells: (1,4) (1,5)
+  6   2 cells: (1,6) (2,6)
-  2   2 cells: (1,7) (2,7)
* 24   2 cells: (2,2) (2,3)
+ 15   2 cells: (2,4) (2,5)
/  3   2 cells: (3,0) (3,1)
* 64   3 cells: (3,2) (4,2) (4,1)
* 70   3 cells: (3,3) (3,4) (3,5)
* 24   2 cells: (3,6) (3,7)
/  3   2 cells: (4,0) (5,0)
* 15   2 cells: (4,3) (5,3)
/  2   2 cells: (4,4) (4,5)
* 21   3 cells: (4,6) (4,7) (5,7)
+ 11   2 cells: (5,1) (5,2)
+ 13   3 cells: (5,4) (6,4) (6,3)
-  1   2 cells: (5,5) (5,6)
* 35   2 cells: (6,0) (6,1)
+  7   2 cells: (6,2) (7,2)
/  2   2 cells: (6,5) (7,5)
-  1   2 cells: (6,6) (7,6)
-  5   2 cells: (6,7) (7,7)
/  2   2 cells: (7,0) (7,1)
-  5   2 cells: (7,3) (7,4)


In [4]:
# Validate that we have the right number of total cells, and unique cells
puzzle.board_size**2, sum(len(c.cells) for c in puzzle.cages), len(set.union(*(set(c.cells) for c in puzzle.cages)))

(64, 64, 64)

In [5]:
BOARD_SIZE = puzzle.board_size

# convenience func
def get_nums():
    return range(1, BOARD_SIZE+1)

# create a grid of possible values
possible_values = {Cell(row, col): set(get_nums()) for row in range(BOARD_SIZE) for col in range(BOARD_SIZE)}
possible_values


{Cell(row=0, col=0): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=1): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=2): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=3): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=4): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=5): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=6): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=0, col=7): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=0): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=1): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=2): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=3): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=4): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=5): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=6): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=1, col=7): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=2, col=0): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=2, col=1): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=2, col=2): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=2, col=3): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=2, col=4): {1, 2, 3, 4, 5, 6, 7, 8},
 Cell(row=2, 

In [6]:
# Pretty-printing the current state
def print_cell(nums):
    return "".join([str(i) if i in nums else " " for i in get_nums()])

def print_grid(values):
    for row in range(BOARD_SIZE):
        print ("|".join([print_cell(values[(row, col)]) for col in range(BOARD_SIZE)]))

print_grid(possible_values)


12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678
12345678|12345678|12345678|12345678|12345678|12345678|12345678|12345678


In [7]:
# For a cage, enumerate all the sets of numbers that could meet the arithmetic constraint
def enum_cage_solutions(cage):
    solutions = {}
    # Pass the operator, size and value
    # Use pattern matching to validate the right number of cells for an operator
    match (cage.op, len(cage.cells), cage.val):
        case (op.NULL, 1, val):
            solutions = [[val]]
        case (op.DIV, 2, val):
            solutions = [(i, i*val) for i in range(1, int(BOARD_SIZE/val)+1)]
        case (op.SUB, 2, val):
            solutions = [(i, i+val) for i in range(1, BOARD_SIZE-val+1)]
        case (op.PROD, size, val):
            factors = (i for i in get_nums() if val%i == 0)
            combos = itertools.product(factors, repeat=size)
            possibles = [tuple(sorted(combo)) for combo in combos if math.prod(combo)==val]
            solutions = list(set(possibles))
        case (op.SUM, size, val):
            possible_nums = get_nums()
            combos = itertools.product(possible_nums, repeat=size)
            possibles = [tuple(sorted(combo)) for combo in combos if sum(combo)==val]
            solutions = list(set(possibles))
        case _:
            raise RuntimeError(f"Unrecognized operator, or wrong size for operator in {cage}")
    return solutions

In [8]:
[enum_cage_solutions(s) for s in puzzle.cages]

[[(1, 8)],
 [(3, 8), (4, 6)],
 [(6, 6, 6), (5, 5, 8), (4, 6, 8), (4, 7, 7), (5, 6, 7), (2, 8, 8), (3, 7, 8)],
 [(1, 3), (2, 6)],
 [(2, 5)],
 [(3, 5, 7)],
 [(1, 3), (2, 6)],
 [(2, 4), (3, 3), (1, 5)],
 [(1, 3), (2, 4), (3, 5), (4, 6), (5, 7), (6, 8)],
 [(3, 8), (4, 6)],
 [(7, 8)],
 [(1, 3), (2, 6)],
 [(4, 4, 4), (1, 8, 8), (2, 4, 8)],
 [(2, 5, 7)],
 [(3, 8), (4, 6)],
 [(1, 3), (2, 6)],
 [(3, 5)],
 [(1, 2), (2, 4), (3, 6), (4, 8)],
 [(1, 3, 7)],
 [(3, 8), (5, 6), (4, 7)],
 [(3, 4, 6),
  (1, 5, 7),
  (3, 3, 7),
  (1, 6, 6),
  (4, 4, 5),
  (2, 4, 7),
  (2, 3, 8),
  (3, 5, 5),
  (2, 5, 6),
  (1, 4, 8)],
 [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8)],
 [(5, 7)],
 [(1, 6), (2, 5), (3, 4)],
 [(1, 2), (2, 4), (3, 6), (4, 8)],
 [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8)],
 [(1, 6), (2, 7), (3, 8)],
 [(1, 2), (2, 4), (3, 6), (4, 8)],
 [(1, 6), (2, 7), (3, 8)]]

In [9]:
# Apply the cage math
# For each cage, compute the possible values that can fill the cage,
# and restrict the global possible values to those values
def do_cage_math_logic():
    for cage in puzzle.cages:
        sols = enum_cage_solutions(cage)
        nums = set.union(*[set(sol) for sol in sols])
        for cell in cage.cells:
            possible_values[cell] = possible_values[cell].intersection(nums)

do_cage_math_logic()
print_grid(possible_values)

1      8|  34 6 8|  34 6 8| 2345678|123  6  |123  6  | 2  5   | 2  5   
1      8|  3 5 7 | 2345678| 2345678|123  6  |123  6  |12345   |12345678
  3 5 7 |  3 5 7 |  34 6 8|  34 6 8|      78|      78|12345   |12345678
123  6  |123  6  |12 4   8| 2  5 7 | 2  5 7 | 2  5 7 |  34 6 8|  34 6 8
123  6  |12 4   8|12 4   8|  3 5   |1234 6 8|1234 6 8|1 3   7 |1 3   7 
123  6  |  345678|  345678|  3 5   |12345678|12345678|12345678|1 3   7 
    5 7 |    5 7 |123456  |12345678|12345678|1234 6 8|12345678|123  678
1234 6 8|1234 6 8|123456  |123  678|123  678|1234 6 8|12345678|123  678


In [10]:
# This does the pigeonhole logic of one number per row/column
# The logic is easy when a cell has only one possible value
# But it can be generalized; if a row has two pairs with two values, those two values can't be elsewhere
# Eg the ' 23' in two cells of row 0 means that there can't be a 2 or 3 elsewhere in row 0
# And so on for 3 sets of size 3, etc
# TODO generalize this to columns
def do_pigeonhole_logic():
    for row in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {col: possible_values[(row, col)] for col in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for col in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

    # For now copy/paste for cols
    for col in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {row: possible_values[(row, col)] for row in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for row in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

do_pigeonhole_logic()
print_grid(possible_values)

1      8|  34 6 8|  34 6 8|   4 678|1 3  6  |1 3  6  | 2  5   | 2  5   
1      8|  3 5 7 | 2345678| 2 4 678|123  6  |123  6  |12345   |12345678
  3 5   |  3 5   |  34 6  |   4 6  |      78|      78|12345   |123456  
  3  6  |1 3  6  |1  4   8| 2    7 | 2  5 7 | 2  5 7 |  34 6 8|  34 6 8
 23  6  |12 4   8|12 4   8|  3 5   |1234 6 8|1234 6 8|1 3   7 |1 3   7 
 23  6  |  345678|  345678|  3 5   |12345678|12345678|12345678|1 3   7 
    5 7 |    5 7 |1234 6  |12 4 6 8|1234 6 8|1234 6 8|1234 6 8|123  6 8
 234 6  |1234 6 8|123456  |12   678|123  678|1234 6 8|12345678|123  678


In [11]:
# another iteration
do_pigeonhole_logic()
print_grid(possible_values)

1      8|  34 6 8|  34 6 8|   4 678|1 3  6  |1 3  6  | 2  5   | 2  5   
1      8|  3 5 7 | 2345678| 2 4 678|123  6  |123  6  |12345   |12345678
  3 5   |  3 5   |   4 6  |   4 6  |      78|      78|12 4    |12 4 6  
  3  6  |1 3  6  |1  4   8| 2    7 | 2  5 7 | 2  5 7 |  34 6 8|  34 6 8
 23  6  |12 4   8|12 4   8|  3 5   |1234 6 8|1234 6 8|1 3   7 |1 3   7 
 23  6  |  345678|  345678|  3 5   |12345678|12345678|12345678|1 3   7 
    5 7 |    5 7 |1234 6  |12 4 6 8|1234 6 8|1234 6 8|1234 6 8|123  6 8
 234 6  |1234 6 8|123456  |12   678|123  678|1234 6 8|12345678|123  678


In [12]:
# a few more iterations
[do_pigeonhole_logic() for _ in range(5)]
print_grid(possible_values)

1      8|  34 6 8|  34 6 8|   4 678|1 3  6  |1 3  6  | 2  5   | 2  5   
1      8|  3 5 7 | 2345678| 2 4 678|123  6  |123  6  |12345   |12345678
  3 5   |  3 5   |   4 6  |   4 6  |      78|      78|12      |12      
  3  6  |1 3  6  |1  4   8| 2    7 | 2  5 7 | 2  5 7 |  34 6 8|  34 6 8
 23  6  |12 4   8|12 4   8|  3 5   |1234 6 8|1234 6 8|1 3   7 |1 3   7 
 23  6  |  345678|  345678|  3 5   |12345678|12345678|12345678|1 3   7 
    5 7 |    5 7 |1234 6  |12 4 6 8|1234 6 8|1234 6 8|1234 6 8|123  6 8
 234 6  |1234 6 8|123456  |12   678|123  678|1234 6 8|12345678|123  678


In [13]:
# TODO the cage math logic is pretty basic now
# I could expand it to see which sets are actually possible based on current state
# E.g., if a valid set is {1, 2 ,3}, but we know that 3 can't be in any of the three cells,
# we can eliminate that set

# TODO iteratively run the pigeonhole and math logic until we solve the problem (or converge without a solution)